In [9]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [2]:
from langchain_openai import ChatOpenAI

In [10]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

In [5]:
load_dotenv()
KEY  = os.getenv("OPENAI_API_KEY")

In [6]:
print(KEY)

KEY fetched


In [7]:
llm = ChatOpenAI(openai_api_key = KEY, model="gpt-3.5-turbo", temperature=0.5)

In [8]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002656F3DF3D0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002656F4E97F0>, temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [12]:
response_json = {
    1:{
        "mcq":"question",
        "options":{
            "a":"option1",
            "b":"option2",
            "c":"option3"
        },
        "answer":"correct answer"
    },
    2:{
        "mcq":"question",
        "options":{
            "a":"option1",
            "b":"option2",
            "c":"option3"
        },
        "answer":"correct answer"
    },
}

In [13]:
TEMPLATE = """
Text:{text}
You are an expert mcq maker. Given the above text, it is your job to creat {number} of 
questions for {subject} students in {tone}.ensure to make {number} of question without repeatition.
make sure to format your reponse like response_json blow.
### response_json
{response_json}
"""

In [14]:
quiz_gen_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [15]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_gen_prompt,output_key="quiz", verbose=True)

In [16]:
TEMPLATE2 = """
you are an expert english grammarian and writer. given a mcq for {subject} student.
Evaluate the complexity and update the questions needs to change to match with the abilities of students.
mcq quiz:{quiz}.
check from an expert english writer of the above quiz:
"""

In [17]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [18]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [19]:
gen_eval_chain = SequentialChain(chains=[quiz_chain,review_chain], input_variables=["text","number","subject","tone","response_json"],
                                  output_variables=["quiz","review"], verbose=True)

In [20]:
TEXT = os.getenv("TEXT")

In [23]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "simple"

In [31]:
with get_openai_callback() as cb:
    try:
        response = gen_eval_chain(
            {
                "text" : TEXT,
                "number" : NUMBER,
                "subject" : SUBJECT,
                "tone" : TONE,
                "response_json" : json.dumps(response_json)
            }
        )
    except:
        response = "Result fetched. API KEY required to display the output."
print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:None
You are an expert mcq maker. Given the above text, it is your job to creat 5 of 
questions for Machine Learning students in simple.ensure to make 5 of question without repeatition.
make sure to format your reponse like response_json blow.
### response_json
{"1": {"mcq": "question", "options": {"a": "option1", "b": "option2", "c": "option3"}, "answer": "correct answer"}, "2": {"mcq": "question", "options": {"a": "option1", "b": "option2", "c": "option3"}, "answer": "correct answer"}}

Result fetched. API KEY required to display the output.


In [33]:
print(cb.prompt_tokens)

0
